# Time Analysis

This notebook contains time analysis for statistical sampling based estimators vs. the proposed framework.

In [1]:
import sys
sys.path.insert(0, './../src/')

In [2]:
from time import perf_counter
from estimators import GEE, AE, UJ2A, SGD, PA, RLS
from visualization import time_estimate, time_fit, time_predict, time_features_labels, df_to_pdf
from preprocessing import TraceStats
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FormatStrFormatter

In [3]:
statistical = [GEE('GEE'), AE('AE'), UJ2A('UJ2A')]
features = ['f_1' , 'f_2', 'f_3', 'avg_pkt_len', 'syn_count']
online_ml = [SGD, PA, RLS]
online_ml_names = ['SGD', 'PA', 'RLS']
sampling_rates = [0.1, 0.2, 0.3, 0.4, 0.5]

# load ts
tss = [TraceStats.load('./../data/time_analysis_caida-2016_100K_%.4f.pickle' % sampling_rate) for sampling_rate in sampling_rates]
tss[0].batch_count

446

### Estimation Time

In [4]:
# load df
df_estimate = pd.read_pickle('time_estimate.pickle')
df_to_pdf(df_estimate, 'time_estimate', print_index=False, digit_round=None,
          comma_separated_columns=['Mean Sample Size'], gen_latex=True)
df_estimate

,Mean Sample Size,GEE,AE,UJ2A
1,"10,000",6.20e-06,2.72e-03,4.86e-05
2,"20,000",7.69e-06,4.06e-03,6.99e-05
3,"30,000",9.01e-06,5.33e-03,8.95e-05
4,"40,000",1.03e-05,6.33e-03,1.08e-04
5,"50,000",1.14e-05,7.44e-03,1.34e-04


### Training Time

In [5]:
df_fit = pd.read_pickle('time_fit.pickle')
df_to_pdf(df_fit, 'time_fit', print_index=False, digit_round=None,
          comma_separated_columns=['Mean Sample Size'], gen_latex=True)
df_fit

,Mean Sample Size,SGD,PA,RLS
1,"10,000",2.38e-05,3.70e-05,2.66e-05
2,"20,000",2.08e-05,3.49e-05,2.66e-05
3,"30,000",2.09e-05,3.49e-05,3.06e-05
4,"40,000",2.13e-05,3.51e-05,2.65e-05
5,"50,000",2.82e-05,3.52e-05,2.62e-05


### Prediction Time

In [6]:
df_predict = pd.read_pickle('time_predict.pickle')
df_to_pdf(df_predict, 'time_predict', print_index=False, digit_round=None,
         comma_separated_columns=['Mean Sample Size'], gen_latex=True)
df_predict

,Mean Sample Size,SGD,PA,RLS
1,"10,000",8.81e-06,8.52e-06,3.55e-06
2,"20,000",7.68e-06,8.20e-06,3.62e-06
3,"30,000",8.23e-06,8.36e-06,3.70e-06
4,"40,000",8.15e-06,8.36e-06,3.53e-06
5,"50,000",8.40e-06,9.25e-06,3.91e-06


### Featurization Time

In [4]:
df_features_labels = time_features_labels(tss)
df_features_labels = df_features_labels[['Sampling Rate', 'Featurization (mean)', 'Labeling (mean)']]
df_to_pdf(df_features_labels, 'time_features_labels', print_index=False, digit_round=2,
          gen_latex=True)
df_features_labels

,Sampling Rate,Featurization (mean),Labeling (mean)
0,0.1,0.428401,4.232068
1,0.2,0.848065,4.180708
2,0.3,1.260601,4.163787
3,0.4,1.659300,4.112581
4,0.5,2.074345,4.106532


### Profile CPU usage of featurization time

In [6]:
from preprocessing import pcap_to_df, BatchStats
from functools import partial
import numpy as np
from xxhash import xxh32_hexdigest, xxh64_hexdigest

In [7]:
parser_func = BatchStats.from_pcap
parser = partial(parser_func, sampling_rate=0.1)

In [8]:
trace_df = pcap_to_df('./../data/caida-2016_100K_00000_20160317145911.pcap')

**ORIGINAL CALL**

In [9]:
%%prun
trace_df.apply(lambda x: hash((x['ip.src'], x['ip.dst'], x['tcp.port'], x['udp.port'], x['ip.proto'])), axis=1)

We can see that a lot of time is spent on getting stuff from memory. We can assume that data would be more easily accessible in an optimized implementation.

**OPTIMIZED CALL**

In [24]:
# convert the columns to be hashed to an ordered numpy array

arr = trace_df[['ip.src', 'ip.dst', 'tcp.port', 'udp.port','ip.proto']].to_numpy(str)
arr = np.asarray(arr, order='C')

CPU times: user 245 ms, sys: 24.5 ms, total: 269 ms
Wall time: 268 ms


In [12]:
%%prun
packets = np.apply_along_axis(lambda x:hash(tuple(x)), 1, arr)

Much better, lets check a more suitble hash function (non crypto - optimized for unique values).

In [13]:
%%prun
packets = np.apply_along_axis(xxh32_hexdigest, 1, arr)

Lets find the real times(without profiling)

In [18]:
%%time
packets = trace_df.apply(lambda x: hash((x['ip.src'], x['ip.dst'], x['tcp.port'], x['udp.port'], x['ip.proto'])), axis=1)

CPU times: user 4.84 s, sys: 21.1 ms, total: 4.87 s
Wall time: 4.87 s


In [19]:
len(np.unique(packets))

22399

In [25]:
%%time

# convert the columns to be hashed to an ordered numpy array

arr = trace_df[['ip.src', 'ip.dst', 'tcp.port', 'udp.port','ip.proto']].to_numpy(str)
arr = np.asarray(arr, order='C')

CPU times: user 234 ms, sys: 24.4 ms, total: 258 ms
Wall time: 255 ms


In [20]:
%%time
packets = np.apply_along_axis(lambda x:hash(tuple(x)), 1, arr)

CPU times: user 545 ms, sys: 3.86 ms, total: 549 ms
Wall time: 547 ms


In [21]:
len(np.unique(packets))

22399

In [22]:
%%time
packets = np.apply_along_axis(xxh32_hexdigest, 1, arr)

CPU times: user 435 ms, sys: 3.27 ms, total: 438 ms
Wall time: 437 ms


In [23]:
len(np.unique(packets))

22399